# Feature Engineering

Transform raw data into powerful predictors using MKYZ's feature engineering tools.

In [1]:
import mkyz
import pandas as pd
mkyz.init()

mkyz package initialized. Version: 0.2.0
mkyz package initialized. Version: 0.2.0


## 1. Load Housing Data

We will use the Boston Housing dataset.

In [2]:
df = mkyz.load_data('data/boston_housing.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


## 2. Create Interaction Features

Automatically generate interactions between numerical features.

In [3]:
fe = mkyz.FeatureEngineer()

# Create interactions between specific columns
df = fe.create_interaction_features(df, columns=['rm', 'lstat'])

# Check for new columns
print([c for c in df.columns if '_x_' in c])

['rm_x_lstat']


## 3. Polynomial Features

Capture non-linear relationships.

In [4]:
df = fe.create_polynomial_features(df, columns=['crim', 'tax'], degree=2)
print("Polynomial features created.")

Polynomial features created.


## 4. Feature Selection

Select the most relevant features for the target variable 'medv'.

In [5]:
X = df.drop('medv', axis=1)
y = df['medv']

# Select top 5 features
selected = mkyz.select_features(X, y, n_features=5, method='f_regression')
print("Top 5 Features:", selected)

TypeError: select_features() got an unexpected keyword argument 'n_features'